In [3]:
def remove_zeroes(L):
    R = L[0].parent()
    L = list(L)
    while len(L) > 0 and L[-1] == R(0):
        L.pop()
    return L

In [1]:
def poly_add(A, B):
    R = A[0].parent()
    n = max(len(A), len(B))
    res = [R(0)] * n
    for i in range(n):
        a = A[i] if i < len(A) else 0
        b = B[i] if i < len(B) else 0
        res[i] += a + b
    res = remove_zeroes(res)
    return res

In [2]:
def poly_mult(A, B):
    R = A[0].parent()
    res = [R(0)] * (len(A) + len(B) - 1)
    for i in range(len(A)):
        for j in range(len(B)):
            res[i + j] += A[i] * B[j]
    res = remove_zeroes(res)
    return res


In [4]:
def list_to_poly(L):
    R = L[0].parent()
    # cree le polynome associe a une liste
    x = R['x'].gen()
    return sum(L[i] * x^i for i in range(len(L)))

def random_poly_list(deg, R):
    return [R.random_element() for _ in range(deg + 1)]

def test_operations(R, deg1, deg2):
    A = random_poly_list(deg1, R)
    B = random_poly_list(deg2, R)

    PR = PolynomialRing(R, 'x')

    PA = list_to_poly(A)
    PB = list_to_poly(B)

    C_add = poly_add(A, B)
    C_mul = poly_mult(A, B)

    assert list_to_poly(C_add) == PA + PB
    assert list_to_poly(C_mul) == PA * PB

    print(f"OK pour degrés {deg1} et {deg2} sur {R}")

In [ ]:
def run_tests_1():
    rings = [GF(17), ZZ, Integers(100)]
    degrees = [10, 100, 1000]

    for R in rings:
        # tests degres identiques
        for d in degrees:
            test_operations(R, d, d)
        # tests degres differents
        for i in range(3):
            for j in range(i + 1, 3):
                test_operations(R, degrees[i], degrees[j])

# run_tests_1()

OK pour degrés 10 et 10 sur Finite Field of size 17
OK pour degrés 100 et 100 sur Finite Field of size 17
OK pour degrés 1000 et 1000 sur Finite Field of size 17
OK pour degrés 10 et 100 sur Finite Field of size 17
OK pour degrés 10 et 1000 sur Finite Field of size 17
OK pour degrés 100 et 1000 sur Finite Field of size 17
OK pour degrés 10 et 10 sur Integer Ring
OK pour degrés 100 et 100 sur Integer Ring
OK pour degrés 1000 et 1000 sur Integer Ring
OK pour degrés 10 et 100 sur Integer Ring
OK pour degrés 10 et 1000 sur Integer Ring
OK pour degrés 100 et 1000 sur Integer Ring
OK pour degrés 10 et 10 sur Ring of integers modulo 100
OK pour degrés 100 et 100 sur Ring of integers modulo 100
OK pour degrés 1000 et 1000 sur Ring of integers modulo 100
OK pour degrés 10 et 100 sur Ring of integers modulo 100
OK pour degrés 10 et 1000 sur Ring of integers modulo 100
OK pour degrés 100 et 1000 sur Ring of integers modulo 100


In [ ]:
def fill_to_n(x, n):
    if x == []:
        return [0] * n
    R = x[0].parent()
    return x + [R(0)] * (n - len(x))

def karatsuba(a, b):
    # --- cas vide ---
    if not a or not b:
        return []
    # ----------------
    
    la = len(a)
    lb = len(b)
    
    # --- cas constant ---
    if la == 1:
        return [a[0] * bi for bi in b]
    if lb == 1:
        return [b[0] * ai for ai in a]
    # --------------------
    
    R = a[0].parent()
    n = max(len(a), len(b))
    m = n // 2

    # --- on separe les polynomes en deux ---
    a0 = a[:m]
    a1 = a[m:]
    b0 = b[:m]
    b1 = b[m:]
    # ---------------------------------------
    
    # --- on calcule les trois produits ---
    a0a1 = poly_add(a0, a1)
    b0b1 = poly_add(b0, b1)
    
    z0 = karatsuba(a0, b0)
    z2 = karatsuba(a1, b1)
    z1 = karatsuba(a0a1, b0b1)
    # -------------------------------------
    
    # --- on combine les resultats ---
    result = [R(0)] * (2 * n)
    L = max(len(z0), len(z1), len(z2))
    z0 = fill_to_n(z0, L)
    z1 = fill_to_n(z1, L)
    z2 = fill_to_n(z2, L)
    for i in range(L):
        z1[i] -= z0[i] + z2[i]

    # --- partie de z0 et z2 ---
    for i, v in enumerate(z0):
        result[i] += v

    for i, v in enumerate(z2):
        result[i + 2*m] += v
    
    # --- partie de z1 ---
    for i, v in enumerate(z1):
        result[i + m] += v
    # -------------------------------------

    # --- on retire les zeros a la fin ---
    while result and result[-1] == R(0):
        result.pop()
    # ------------------------------------
    
    return result

In [7]:
R = GF(5)

a = [1, 2, 3]
b = [2, 1]

karatsuba(a, b)

[2, 5, 8, 3]

In [8]:
# EXERCICE 8

In [9]:
def vect_mult(p, q):
    R = p[0].parent()
    n_p = len(p)
    n_q = len(q)
    n = max(n_p, n_q)
    
    i = 0
    while len(p) < n:
        p.append(R(0))
    
    while len(q) < n:
        q.append(R(0))
    
    res = R(0)
    for i in range(n):
        res += p[i]* q[i]
    
    return res

R.<x> = QQ[]
V = [
    [1, 1, 1],
    [1, 2, 4],
    [1, 3, 9]
]
var('x1 x2 x3')
y = [x1, x2, x3]

assert vect_mult(V[0], y) == x1 + 1*x2 + 1^2 * x3
assert vect_mult(V[1], y) == x1 + 2*x2 + 2^2 * x3
assert vect_mult(V[2], y) == x1 + 3*x2 + 3^2 * x3

print("On a bien V[i] * y qui donne le polynome evalue en certains points")

On a bien V[i] * y qui donne le polynome evalue en certains points


In [10]:
# on essaie maintenant la meme chose sur le polynome P(x) = 1 + 3x - 5x^2 dans Z[x]:
R.<x> = ZZ[]
V = [
    [1, 1, 1],
    [1, 2, 4],
    [1, 3, 9]
]

y = [R(1), R(3), R(-5)]

assert vect_mult(V[0], y) == y[0] + 1*y[1] + 1^2 * y[2]
assert vect_mult(V[1], y) == y[0] + 2*y[1] + 2^2 * y[2]
assert vect_mult(V[2], y) == y[0] + 3*y[1] + 3^2 * y[2]
for i in range(3):
    print(vect_mult(V[i], y))
print("On a bien V[i] * y qui donne les bons resultats pour le polynome:\nP(x) = 1 + 3x 5x^2")

-1
-13
-35
On a bien V[i] * y qui donne les bons resultats pour le polynome:
P(x) = 1 + 3x 5x^2


In [11]:
# Par le cours on sait qu'on peut calculer V^{-1} sans inverser la matrice, 
# on sait que les colonnes correspondent aux L_i, et ces memes L_i on peut les calculer comme suit
# M = prod (x-x_i)
# M_i = M/(x-x_i)
# L_i = M_i/prod(...)
### a completer

In [12]:
# exo 8-4
def evaluation(P, pl):
    R = P[0].parent()
    deg_P = len(P) - 1
    nb_coeffs = len(pl)
    res = []
    for j in range(nb_coeffs):
        tmp = 0
        for i in range(deg_P + 1):
            tmp += P[i] * pl[j]^i
        res.append(tmp)
    return res

y = [1, 2, 3]
evaluation([R(1), R(3), R(-5)], y)

[-1, -13, -35]

In [13]:
# exo 8-5
def interpolation(couples):
    alphas, y = couples
    degp1 = len(alphas)
    # on commence par calculer M
    M(x) = 1
    for i in range(degp1):
        M(x) = poly_mult(M(x), (x- alphas[i]))
    # on fait une matrice pour les M_i
    MIS = [O] * degp1
    for i in range(degp1):
        MIS[i] = poly_div(M(x), (x-alphas[i]))
    # on recalcule les L_i
    LIS = [0] * degp1
    for i in range(degp1):
        LIS[i] = MIS[i](x) / MIS[i](alphas[i])
    # on remultiplie par les y_i pour avoir le polynome final
    P(x) = 0
    for i in range(degp1):
        P += LIS[i](x) * y[i]
    

In [15]:
# exo 10 partie 1
R = GF(13)

def get_vandermonde(omega, p):
    R = omega.parent()
    V = matrix(R, p, p, lambda i, j: omega^(i*j))
    return V

V1 = get_vandermonde(R(8), 4)
# l'inverse de omega est l'element x tel que omega x = 1, soit 4-1=3 donc 8^3 car omega est 3 primitif -> 5
# on peut aussi juste le trouver avec elem^(-1)
print(V1)
print("")
V2 = get_vandermonde(R(5), 4)
print(V2)

[ 1  1  1  1]
[ 1  8 12  5]
[ 1 12  1 12]
[ 1  5 12  8]

[ 1  1  1  1]
[ 1  5 12  8]
[ 1 12  1 12]
[ 1  8 12  5]


In [24]:
# exo 10 partie 2

# 2 est une racine 5 primitive de l'unite dans GF31 , son inverse est alors 2^4  16
R = GF(31)
omega = R(2)
omega_inv = omega^(-1)
g = R.multiplicative_generator()
omega = g^6
prim = 5

V1 = get_vandermonde(omega, prim)
#print(V1)
#print("")
V2 = get_vandermonde(omega_inv, prim)
#print(V2)



def multiplication_FFT(P, Q, omega, n):
    R = omega.parent()
    
    # on remplit de zeros
    P1 = vector(R, list(P) + [0]*(n - len(P)))
    Q1 = vector(R, list(Q) + [0]*(n - len(Q)))

    
    V = get_vandermonde(omega, n)
    V_inv = get_vandermonde(omega^(-1), n)
    
    # evaluation 1
    P2 = V * P1
    Q2 = V * Q1
    
    # multiplication des valeurs
    res_tmp = vector(R, [P2[i] * Q2[i] for i in range(n)])
    
    # evaluation 2
    res = (1/R(n)) * (V_inv * res_tmp)
    
    res = remove_zeroes(res)
    return res



R = GF(31)
# 5e racine primitive, 6 = (31 - 1)/5 (pour GF)
#g = R.multiplicative_generator()
#omega = g^6   
# sinon a la main 2^5 = 1
omega = R(2)

res1 = multiplication_FFT(vector(R, [1, 11]),
                          vector(R, [18, 5]),
                          omega, 5)
true_res1 = karatsuba([R(1), R(11)], [R(18), R(5)])

res2 = multiplication_FFT(vector(R, [29, 11, 16]),
                          vector (R, [18, 0, 5]),
                          omega, 5)
true_res2 = karatsuba([R(29), R(11), R(16)], [R(18), R(0), R(5)])

assert res1 == true_res1
assert res2 == true_res2

print("La multiplication FFT donne bien le bon resultat")

La multiplication FFT donne bien le bon resultat


In [ ]:
######################


def FFT(P, omega, n):
    # on va faire une FFT sur le polynome P, on suppose que n est une puissance de 2 superieure au degre de P
    # on suppose aussi que omega est une racine n-ieme primitive de l'unite
    R = omega.parent()
    if n == 1:
        return P
    if len(P) <= 1:
        return P
    print(f"FFT sur {P} avec omega = {omega} et n = {n}")
    
    P_pair = [P[i] for i in range(0, len(P), 2)]
    P_impair = [P[i] for i in range(1, len(P), 2)]
    # on fait la FFT sur les deux moities
    FFT_pair = FFT(P_pair, omega^2, n//2)
    FFT_impair = FFT(P_impair, omega^2, n//2)
    
    # on combine les resultats
    res = [R(0)] * n
    for i in range(n//2):
        res[i] = FFT_pair[i] + omega^i * FFT_impair[i]
        res[i + n//2] = FFT_pair[i] - omega^i * FFT_impair[i]
    return res

# on teste simplement ici FFT
R = GF(11)
g = R.multiplicative_generator()
omega = euler_phi(11)
t = primitive_root(11)
print("omega = ", omega)
print("t = ", t)
testrr = FFT([R(3), R(2), R(6)], omega, t)
print("testrr = ", testrr)
# ce polynome est 3 + 2x + 6x^2, à evaluer en omega^0 jusqu'a t

###
###

######################

omega =  10
t =  2
FFT sur [3, 2, 6] avec omega = 10 et n = 2
[5, 1]


In [17]:
def pow2sup(x):  
    return 1 if x == 0 else 2**(x - 1).bit_length()

def random_poly_list_taille(taille, R):
    return [R.random_element() for _ in range(taille)]

def test_fft(R, taille1, taille2):
    t = taille1 + taille2 + 1
    t = pow2sup(t)
    # on garanti que t est la plus petite puissance de 2 au dessus de t
    A = random_poly_list_taille(taille1, R)
    B = random_poly_list_taille(taille2, R)
    #print(f"t = {t}")
    #print(f"A = {A}")
    #print(f"B = {B}")
    g = R.multiplicative_generator()
    omega = g^((257-1)/t)
    #print(omega)
    #print(omega^t)

    PR = PolynomialRing(R, 'x')

    res_FFT = multiplication_FFT(vector(R, A), vector(R, B), omega, t)
    res_tru = karatsuba(A, B)

    assert res_FFT == res_tru

    print(f"OK pour tailles {taille1} et {taille2} sur {R}")

rings = [GF(257)]
mx = sqrt(pow2sup(257-1))/2-1
print(f"on va aller jusqu'a {mx}")
for R in rings:
    # tests degres identiques
    for k in range(1,mx):
        test_fft(R, 2^k, 2^k)
    # tests degres differents
    for i in range(1, mx):
        for j in range(i + 1, mx):
            test_fft(R, 2^i, 2^j)

on va aller jusqu'a 7
OK pour tailles 2 et 2 sur Finite Field of size 257
OK pour tailles 4 et 4 sur Finite Field of size 257
OK pour tailles 8 et 8 sur Finite Field of size 257
OK pour tailles 16 et 16 sur Finite Field of size 257
OK pour tailles 32 et 32 sur Finite Field of size 257
OK pour tailles 64 et 64 sur Finite Field of size 257
OK pour tailles 2 et 4 sur Finite Field of size 257
OK pour tailles 2 et 8 sur Finite Field of size 257
OK pour tailles 2 et 16 sur Finite Field of size 257
OK pour tailles 2 et 32 sur Finite Field of size 257
OK pour tailles 2 et 64 sur Finite Field of size 257
OK pour tailles 4 et 8 sur Finite Field of size 257
OK pour tailles 4 et 16 sur Finite Field of size 257
OK pour tailles 4 et 32 sur Finite Field of size 257
OK pour tailles 4 et 64 sur Finite Field of size 257
OK pour tailles 8 et 16 sur Finite Field of size 257
OK pour tailles 8 et 32 sur Finite Field of size 257
OK pour tailles 8 et 64 sur Finite Field of size 257
OK pour tailles 16 et 32 su

In [ ]:
import collections
import math
import time
import random

def test_fft2(R, taille1, taille2):
    t = taille1 + taille2 + 1
    t = pow2sup(t)
    # on garanti que t est la plus petite puissance de 2 au dessus de t
    A = random_poly_list_taille(taille1, R)
    B = random_poly_list_taille(taille2, R)
    #print(f"t = {t}")
    #print(f"A = {A}")
    #print(f"B = {B}")
    g = R.multiplicative_generator()
    omega = g^(pow2sup(30000-1)/2*t)
    #print(omega)
    #print(omega^t)

    start_time = time.time()
    res_FFT = multiplication_FFT(vector(R, A), vector(R, B), omega, t)
    print("%s seconds" % round(time.time() - start_time, 2))
    start_time = time.time()
    res_tru = karatsuba(A, B)
    print("%s seconds" % round(time.time() - start_time, 2))

    assert res_FFT == res_tru

    print(f"OK pour tailles {taille1} et {taille2} sur {R}")

rings = [GF(65537)]
k = 30000
print(f"on va aller jusqu'a {k}")
test_fft2(R, k, k)

# le code ci-dessus plante parce qu'on ne peut pas allouer une matrice de Vandermonde de taille suffisament elevée

on va aller jusqu'a 30000


MemoryError: failed to allocate 4294967296 * 8 bytes